In [14]:
# Step 1: Installation of dependencies and importing libraries

!pip install BeautifulSoup4
!pip install geocoder
!pip install pyzillow

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

In [3]:
# Step 2: Load Zillow and HERE API keys

# Get Zillow API key (https://www.zillow.com/howto/api/APIOverview.htm)
zwsid_key = ''
zillow_data = ZillowWrapper(zwsid_key)

#Get HERE API tokens (https://developer.here.com/)
here_app_id = ''
here_app_code = ''

In [15]:
# Step 3: Get county page for sheriff foreclosure sales, and feed into Beautiful Soup,
#         and grab specific content of interest

page = requests.get('https://www.co.somerset.nj.us/government/elected-officials/sheriff-s-office/divisions/sheriff-sales')
soup = BeautifulSoup(page.content, 'html.parser')
content_box = soup.find('div', attrs={'class': 'content_area'})
name = content_box.text.strip()
lines = []
nb_lines =[]
for x in content_box:
    lines.append(x)
for i in lines:
    if len(i)>1:
        nb_lines.append(i)
y = []
for i in range(2,len(nb_lines)):
     x=(nb_lines[i].contents[1])
     y.append(x.parent.get_text(strip=True).replace(u'\xa0', u' '))#.encode('utf-8'))

In [6]:
# Step 4: For each row with an address entry (ie not sold or cancelled), feed address into individual columns,
#         and attempt to get information on address from Zillow

sales=[]
dates=[]
addresses=[]
zips=[]
zillows=[]
z_home=[]
z_prop=[]
z_bath=[]
z_bed=[]
z_year=[]
z_tax=[]
z_tax_y=[]
count=1
for i in y:
    if (i[23:24]== '1') or (i[23:24]=='2') or (i[23:24]=='3') or (i[23:24]=='4') or (i[23:24]=='5') or (i[23:24]=='6') or (i[23:24]=='7') or (i[23:24]=='8')or (i[23:24]=='9') or (i[23:24]=='0'):
        sales.append(i[6:11])
        dates.append(i[11:19])
        address = (i[23:])+", NJ"
        addresses.append(address)
        
        # Try to get zipcode from address based on HERE's Geocoder API
        try:
            zipcode = geocoder.here(address, app_id=here_app_id, app_code=here_app_code).postal
            zips.append(str(zipcode))
        except IndexError:
            zips.append("zipcode not found")
            pass
        
        # Search address on Zillow
        if(address.count(',')<=4):
            try:
                deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
                result2 = GetDeepSearchResults(deep_search_response)
                zillows.append(result2.home_detail_link)
                z_home.append(result2.home_size)
                z_prop.append(result2.property_size)
                z_bath.append(result2.bathrooms)
                z_bed.append(result2.bedrooms)
                z_year.append(result2.year_built)
                z_tax.append(result2.tax_value)
                z_tax_y.append(result2.tax_year)
            except:
                zillows.append("n/a")
                z_home.append("n/a")
                z_prop.append("n/a")
                z_bath.append("n/a")
                z_bed.append("n/a")
                z_year.append("n/a")
                z_tax.append("n/a")
                z_tax_y.append("n/a")
                pass
            count+=1
        else:
            zillows.append("n/a")
            count+=1

In [7]:
# Step 5: Merge columns into dataframe

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',500) # Set number of visible rows to a maximum of 500
df = pd.DataFrame(columns = ['Sale #','Date','Address','Zip Code','Zillow Link','Home SqFt','Prop SqFt','# Bath','# Bed','Yr Built','Tax Assess($)','Tax Assess(yr)'])
s2 = pd.Series(sales)
df['Sale #']=s2.values
d2 = pd.Series(dates)
df['Date']=d2.values
df['Date']=pd.to_datetime(df.Date) # Convert values in date column to datetime objects
a2 = pd.Series(addresses)
df['Address']=a2.values
z2 = pd.Series(zips)
df['Zip Code']=z2.values
z3 = pd.Series(zillows)
df['Zillow Link']=z3.values
z4 = pd.Series(z_home)
df['Home SqFt']=z4.values
z5 = pd.Series(z_prop)
df['Prop SqFt']=z5.values
z6 = pd.Series(z_bath)
df['# Bath']=z6.values
z7 = pd.Series(z_bed)
df['# Bed']=z7.values
z8 = pd.Series(z_year)
df['Yr Built']=z8.values
z9 = pd.Series(z_tax)
df['Tax Assess($)']=z9.values
z10 = pd.Series(z_tax_y)
df['Tax Assess(yr)']=z10.values

#Sort table by date
df=df.sort_values(by=['Date'])
display(df)

,Sale #,Date,Address,Zip Code,Zillow Link,Home SqFt,Prop SqFt,# Bath,# Bed,Yr Built,Tax Assess($),Tax Assess(yr)
90,13279,2019-01-08,"11 Amerman Lane, Belle Mead, NJ",08502,https://www.zillow.com/homedetails/11-Amerman-Ln-Belle-Mead-NJ-08502/64426594_zpid/,3756,132727,None,None,2004,533000.0,2018
141,13442,2019-01-08,"1037 Route 28, Branchburg, NJ",08876,https://www.zillow.com/homedetails/1037-Route-28-Somerville-NJ-08876/2131953123_zpid/,None,None,None,None,None,None,None
103,13323,2019-01-08,"89 Winding Way, Franklin, NJ",07416,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
144,13445,2019-01-08,"427 Warfield Road, North Plainfield, NJ",07063,https://www.zillow.com/homedetails/427-Warfield-Rd-North-Plainfield-NJ-07063/39908757_zpid/,1362,7800,2.0,3,1949,230000.0,2018
145,13446,2019-01-08,"416 Washington Avenue, Manville, NJ",08835,https://www.zillow.com/homedetails/416-Washington-Ave-Manville-NJ-08835/39896619_zpid/,1902,6000,None,None,1949,264800.0,2018
132,13426,2019-01-08,"109 Gold Street, Green Brook, NJ",08812,https://www.zillow.com/homedetails/109-Gold-St-Green-Brook-NJ-08812/39883486_zpid/,1818,5227,None,None,1950,273000.0,2018
100,13319,2019-01-08,"11 Como Drive, Somerset, NJ",08873,https://www.zillow.com/homedetails/11-Como-Dr-Somerset-NJ-08873/39882498_zpid/,1192,57499,1.0,3,1955,313300.0,2018
85,13237,2019-01-08,"422 -424 Bartine Street, Somerville, NJ",08876,https://www.zillow.com/homedetails/422-Bartine-St-424-Somerville-NJ-08876/39912914_zpid/,1664,7200,None,None,1945,247600.0,2018
88,13260,2019-01-08,"10 Gregory Lane, Franklin Park, NJ",08823,https://www.zillow.com/homedetails/10-Gregory-Ln-Franklin-Park-NJ-08823/39868086_zpid/,1239,None,None,2,1991,184000.0,2018
97,13314,2019-01-08,"33 Brookside Drive, Franklin, NJ",08540,https://www.zillow.com/homedetails/33-Brookside-Dr-Princeton-NJ-08540/55637926_zpid/,3140,11896,2.5,4,2003,596200.0,2018
